<a href="https://colab.research.google.com/github/young878787/speech2text/blob/main/whisper_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 語音轉文字 AI 工具
本工具使用 [OpenAI 的開源工具 Whisper](https://github.com/openai/whisper) 模型, 可以相對精準的將隨語音轉文字。

# (一) 選擇適合的運作環境： T4 GPU
本 Colab 虛擬機器使用為免費、多GPU的環境。已指定 T4 GPU 版本。

若由  Github 直接開啟，可以忽略此說明。

In [14]:
# @title (1) 安裝 whisper
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-acpozn5r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-acpozn5r
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### (2) 掛載雲端硬碟
1. 透過 Coloab 左邊的操作介面掛載
2. 上傳音檔/影像檔到 Google drive
  - 個人偏好在 Google drive 建一個 tmp 資料夾
  - 將音檔上傳到 tmp 資料夾
  - 在 Colab 左邊的掛載介面找到 drive => MyDrive => tmp
  - 點選上載的音檔，按滑鼠右鍵，點選 複製路徑
3. 將複製的路徑貼到轉檔區塊的 filenames 欄位中


In [16]:
# @title ✅ Whisper 語音轉錄 + 自動上傳回 Google Drive { display-mode: "form" }
# @markdown ---
# @markdown 📂 輸入音訊檔案路徑（Google Drive 中）
filename = "/content/drive/MyDrive/tmp/視聴者さん☆ファッションチェック！！！【潤羽るしあ⧸ホロライブ】 (2).m4a"  # @param {type:"string"}

# @markdown 🧠 Whisper 模型大小
model_size = "medium"  # @param ["tiny", "base", "small", "medium", "large-v2"]

# @markdown 🌏 音訊語言代碼（ja=日文、zh=中文、en=英文）
language = "ja"  # @param {type:"string"}

# @markdown ⏱️ 每段切幾分鐘（5~10 較穩定）
segment_minutes = 10  # @param {type:"number"}

# @markdown ---
# @markdown 🔧 安裝必要套件
!pip install -U openai-whisper ffmpeg-python tqdm --quiet

import os
import whisper
import ffmpeg
from tqdm import tqdm
from datetime import timedelta
import shutil

# 取得檔案名稱與輸出資料夾名稱
file_basename = os.path.splitext(os.path.basename(filename))[0]
drive_output_dir = f"/content/drive/MyDrive/tmp/{file_basename}"
os.makedirs(drive_output_dir, exist_ok=True)

# 切割參數
segment_duration = int(segment_minutes * 60)
base_dir = "/content/split_audio"
os.makedirs(base_dir, exist_ok=True)

# --- 1. 使用 ffmpeg 切割音訊 ---
print("🔪 正在切割音訊...")
input_info = ffmpeg.probe(filename)
duration = float(input_info['format']['duration'])
num_segments = int(duration // segment_duration) + 1

for i in range(num_segments):
    start = i * segment_duration
    output_path = os.path.join(base_dir, f"part_{i:03d}.m4a")
    (
        ffmpeg
        .input(filename, ss=start, t=segment_duration)
        .output(output_path, acodec='copy')
        .run(overwrite_output=True, quiet=True)
    )

# --- 2. Whisper 轉錄 ---
print("🧠 載入模型中...")
model = whisper.load_model(model_size)

def format_timestamp(seconds: float):
    return str(timedelta(seconds=int(seconds))).zfill(8) + ",000"

print("🎧 開始辨識音訊，共 {} 段...".format(num_segments))
all_text = ""

for i in tqdm(range(num_segments)):
    part_path = os.path.join(base_dir, f"part_{i:03d}.m4a")
    result = model.transcribe(part_path, language=language)

    # 儲存 .txt
    with open(os.path.join(base_dir, f"part_{i:03d}.txt"), "w", encoding="utf-8") as f:
        f.write(result["text"])

    # 儲存 .srt
    with open(os.path.join(base_dir, f"part_{i:03d}.srt"), "w", encoding="utf-8") as f:
        for j, seg in enumerate(result["segments"], 1):
            start = format_timestamp(seg["start"])
            end = format_timestamp(seg["end"])
            f.write(f"{j}\n{start} --> {end}\n{seg['text']}\n\n")

    all_text += result["text"] + "\n"

# 儲存總文字
final_txt_path = "/content/whisper_all_output.txt"
with open(final_txt_path, "w", encoding="utf-8") as f:
    f.write(all_text)

# --- 3. 自動上傳回 Google Drive ---
print("📤 上傳輸出檔到 Google Drive 中...")
for file in os.listdir(base_dir):
    shutil.copy(os.path.join(base_dir, file), drive_output_dir)

shutil.copy(final_txt_path, os.path.join(drive_output_dir, "whisper_all_output.txt"))

# --- Done ---
print("✅ 所有轉錄完成，已上傳到 Google Drive：")
print(f"📁 資料夾路徑：{drive_output_dir}")


🔪 正在切割音訊...
🧠 載入模型中...
⏳ 正在轉檔中... 請稍候（防止閒置中）
🎧 開始轉錄音訊，共 30 段...


  0%|          | 0/30 [00:00<?, ?it/s]

⏳ 正在轉檔中... 請稍候（防止閒置中）
⏳ 正在轉檔中... 請稍候（防止閒置中）
⏳ 正在轉檔中... 請稍候（防止閒置中）


  3%|▎         | 1/30 [02:52<1:23:09, 172.03s/it]

⏳ 正在轉檔中... 請稍候（防止閒置中）
⏳ 正在轉檔中... 請稍候（防止閒置中）


...

⏳ 正在轉檔中... 請稍候（防止閒置中）


 97%|█████████▋| 29/30 [1:05:58<02:41, 161.89s/it]

⏳ 正在轉檔中... 請稍候（防止閒置中）


100%|██████████| 30/30 [1:06:50<00:00, 133.68s/it]

✅ 全部完成！字幕與文字檔案已輸出到：
📁 分段資料夾：/content/split_audio
📄 總文字檔：/content/whisper_all_output.txt


# (二) 取得法說會文字
## 1. 請先執行 (1) 安裝 whisper
## 2. 再執行 (4) 法說會逐字稿, ....

In [ ]:
# @title (4) 法說會逐字稿，當影片在 Youtube 可直接使用這個
!pip install yt-dlp

tubeUrl = "https://www.youtube.com/watch?v=Q6sI_eY6sdU" # @param {type:"string"}
import os
from yt_dlp import YoutubeDL
companyName="科技小電報" # @param {type:"string"}
model= "large" # @param {type:"string"}
language = "Chinese" # @param {type:"string"}



filename = companyName+".m4a"
ydl_opts = {'overwrites': True, 'format': 'bestaudio[ext=m4a]', 'outtmpl': filename}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([tubeUrl])

!whisper "{filename}" --model {model} --language {language}

from google.colab import files
exts=["txt","srt","tsv","vtt"]
for ext in exts:
  files.download('{}.{}'.format(companyName,ext))